This project is the beginning of my exploration into the field of algorithmic trading. I began by learning about the IEX Cloud API, which provides financial data. Since this was my first time, I used the IEX sandbox for free experimentation (with guidance from https://www.youtube.com/watch?v=xfzGZB4HhEE). 

I experiemented with momentum trading, a strategy which evaluates portfolio decisions based on fast growing stocks, with some analysis done to determine potential options with characteristics that could sustain this growth.

We take a relatively technical approach, as we look at the returns over One-Year, Six-Months, Three-Months, and One-Month and calculate a score to find higher quality momentum stocks (one that have more consistent returns).

In [51]:
import numpy as np
import pandas as pd
from scipy import stats
import math
import requests

In [52]:
# setting up IEX CLOUD API
tickers = pd.read_csv('/Users/sohammahadik/Downloads/sp_500_stocks.csv')
iexcloud_API = 'Tpk_059b97af715d417d9f49f50b51b1c448'
symbol = 'MSFT'
apiurl = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={iexcloud_API}'
symdata = requests.get(apiurl).json()

In [53]:
# divide stocks into groups for batch API calls
def group(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i+n]

symgroups = list(group(tickers['Ticker'], 100))
symstrings = []
for i in range(0, len(symgroups)):
    symstrings.append(','.join(symgroups[i]))

In [54]:
#populate dataframe with corresponding values/cols
indepth_cols = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile',
    'Momentum Score'
]

indepth_df = pd.DataFrame(columns = indepth_cols)
indepth_df

for symstr in symstrings:
    batchAPI = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symstr}&types=price,stats&token={iexcloud_API}'
    data = requests.get(batchAPI).json()
    for sym in symstr.split(','):
        indepth_df = indepth_df.append(
            pd.Series(
            [
                sym,
                data[sym]['price'],
                'N/A',
                data[sym]['stats']['year1ChangePercent'],
                'N/A',
                data[sym]['stats']['month6ChangePercent'],
                'N/A',
                data[sym]['stats']['month3ChangePercent'],
                'N/A',
                data[sym]['stats']['month1ChangePercent'],
                'N/A',
                'N/A'
            ], 
            index = indepth_cols,
            ),
            ignore_index = True
        )

In [55]:
# Calculate overall percentiles of all stocks by interval Price returns
intervals = [
    'One-Year',
    'Six-Month',
    'Three-Month',
    'One-Month'
]

for r in indepth_df.index:
    for interval in intervals:
        delta = f'{interval} Price Return'
        percent = f'{interval} Return Percentile'
        if indepth_df.loc[r, delta] == None:
            indepth_df.loc[r, delta] = 0.0

for r in indepth_df.index:
    for interval in intervals:
        delta = f'{interval} Price Return'
        percent = f'{interval} Return Percentile'
        indepth_df.loc[r, percent] = stats.percentileofscore(indepth_df[delta], indepth_df.loc[r, delta]) 

In [56]:
# calulate momentum score by taking mean of return percentiles
from statistics import mean

for r in indepth_df.index:
    momPercent = []
    for interval in intervals:
        momPercent.append(indepth_df.loc[r, f'{interval} Return Percentile'])
    indepth_df.loc[r, 'Momentum Score'] = mean(momPercent)

In [57]:
# Take top 50 momentum scores
indepth_df.sort_values('Momentum Score', ascending=False, inplace=True)
indepth_df = indepth_df[:50]
indepth_df.reset_index(inplace=True, drop=True)

In [ ]:
def set_size():
    global size
    size = input("Input the asset size your portfolio:")
    try:
        val = float(size)
    except ValueError:
        print("Please imput a number:")
        portfolio_size = input("Input the asset size your portfolio:")

set_size()

In [ ]:

posSize = float(size)/len(indepth_df.index)
for i in indepth_df.index:
    indepth_df.loc[i, 'Number of Shares to Buy'] = math.floor(posSize/indepth_df.loc[i, 'Price'])
    
indepth_df